In [1]:
import os, sys
import rasterio
import argparse
import yaml
import numpy as np
from tqdm import tqdm
from pathlib import Path
import seaborn as sns
import pandas as pd
from argparse import Namespace
import matplotlib.pyplot as plt
os.chdir("/scratch/ewalt/pdm/rs-uncertainty/")

In [2]:
def pjoin(*subs): return Path(os.path.abspath(os.path.join(*subs)))

In [3]:
# load config
args = Namespace()
args.cfg = Path("./config/evaluate_testset/baseline.yaml")
with args.cfg.open("r", encoding="utf-8") as f:
    cfg = yaml.safe_load(f)

In [4]:
cfg

{'data_bands': [1, 2, 3, 4, 5],
 'variable_names': ['P95', 'MeanH', 'Dens', 'Gini', 'Cover'],
 'projects_east': ['346',
  '9',
  '341',
  '354',
  '415',
  '418',
  '416',
  '429',
  '439',
  '560',
  '472',
  '521',
  '498',
  '522',
  '564',
  '764',
  '781',
  '825',
  '796',
  '805',
  '827',
  '891',
  '835',
  '920',
  '959',
  '1023',
  '998',
  '527',
  '477',
  '542',
  '471'],
 'projects_west': ['528', '537', '792', '988', '769'],
 'projects_north': ['819', '909', '896'],
 'pkl_dir': 'data/2023-04-05_18-58-33',
 'prediction_dir': 'results/2023-04-06_10-52-04_baseline',
 'gt_dir': 'assets/data/preprocessed',
 'num_bins': 1000}

In [5]:
projects = cfg["projects_east"]+cfg["projects_west"]+cfg["projects_north"]
# with Path("/scratch/ewalt/pdm/rs-uncertainty/assets/data/pkl/2021-05-18_10-57-45/stats.yaml").open() as f:
#     stats = yaml.safe_load(f)
# labels_mean = np.array(stats["labels_mean"]).reshape(5,1,1)
# labels_std = np.array(stats["labels_std"]).reshape(5,1,1)
with pjoin(cfg["pkl_dir"], "stats.yaml").open("r", encoding="utf-8") as f:
    stats = yaml.safe_load(f)
labels_mean = np.array(stats["labels_stats"]["mean"]).reshape(5,1,1)
labels_std = np.array(stats["labels_stats"]["std"]).reshape(5,1,1)

In [6]:
# iterate and standardize
variables = ['P95', 'MeanH', 'Dens', 'Gini', 'Cover']
def add(arr, source, state, pid):
    arr = arr.reshape(-1,)   
    data["number"].extend(arr.tolist())
    for variable in variables:
        data["variables"].extend([variable for _ in range(int(arr.shape[0]//5))])
    data["sources"].extend([source for _ in range(arr.shape[0])])
    data["states"].extend([state for _ in range(arr.shape[0])])
    data["project_id"].extend([pid for _ in range(arr.shape[0])])
MAX_PROJECTS = 2
counter = 0
data = {
    "number": [],
    "variables": [],
    "sources": [],
    "states": [],
    "project_id": [],
}
for mean_file in tqdm(Path(cfg["prediction_dir"]).glob('*_mean.tif')):
# for mean_file in Path("/scratch/ewalt/pdm/rs-uncertainty/results/2023-04-05_15-46-48").glob('*_mean.tif'):
    counter += 1
    # load data
    project = mean_file.stem.split('_')[0]
    if project not in projects: continue
    with rasterio.open(mean_file) as fh:
        mean = fh.read(fh.indexes)
    with rasterio.open(pjoin(cfg['prediction_dir'], f"{project}_variance.tif")) as fh:
    # with rasterio.open(pjoin("/scratch/ewalt/pdm/rs-uncertainty/results/2023-04-05_15-46-48", f"{project}_variance.tif")) as fh:
        variance = fh.read(fh.indexes)
    with rasterio.open(pjoin(cfg['gt_dir'], f"{project}.tif")) as fh:
        gt = fh.read(fh.indexes)
        gt_mask = fh.read_masks(1).astype(np.bool_)
        gt[2] /= 100
        gt[4] /= 100
    # print(gt.shape, gt_mask.shape, np.isnan(mean).shape)
    # standardize
    variance_after = variance/(labels_std)**2
    mean_after = (mean-labels_mean)/labels_std
    gt_after = (gt-labels_mean)/labels_std 
    # mask
    mean_mask = ~np.isnan(mean).all(0)
    print("mean_mask included in gt_mask?", (gt_mask[mean_mask==False]==False).all())
    print("gt_mask included in mean_mask?", (mean_mask[gt_mask==False]==False).all())
    mask = np.logical_and(~np.isnan(mean).all(0), gt_mask)
    # print(mask.shape)
    mean = mean[:,mask]
    variance = variance[:,mask]
    gt = gt[:,mask]
    mean_after = mean_after[:,mask]
    variance_after = variance_after[:,mask]
    gt_after = gt_after[:,mask]
    # add flatten
    add(mean, "before", "mean", project)
    add(mean_after, "after", "mean", project)
    add(variance, "before", "variance", project)
    add(variance_after, "after", "variance", project)
    add(gt, "before", "gt", project)
    add(gt_after, "after", "gt", project)
    if counter == MAX_PROJECTS: break
data = pd.DataFrame(data)
data.dropna(inplace=True)
data.reset_index(drop=True,inplace=True)

1it [00:00,  1.01it/s]

mean_mask included in gt_mask? False
gt_mask included in mean_mask? True
mean_mask included in gt_mask? False
gt_mask included in mean_mask? True


In [7]:
data

,number,variables,sources,states,project_id
0,8.502655,P95,before,mean,354
1,10.068137,P95,before,mean,354
2,9.986108,P95,before,mean,354
3,9.772894,P95,before,mean,354
4,10.092015,P95,before,mean,354
...,...,...,...,...,...
5736325,-2.572854,Cover,after,gt,439
5736326,-2.572482,Cover,after,gt,439
5736327,-2.570996,Cover,after,gt,439
5736328,-2.572854,Cover,after,gt,439


In [ ]:
states = ["mean", "gt"]
fig, axs = plt.subplots(nrows=len(data.variables.unique()), ncols=len(states), figsize=(20,20))
for i, variable in enumerate(data.variables.unique()):
    for j, state in enumerate(states):
        if state=="variance": continue
        sub = data.copy()
        sub = sub[sub.variables==variable]
        sub = sub[sub.states==state]
        sns.histplot(data=sub, x="number", hue="sources", kde=True, ax=axs[i,j])
        axs[i,j].set_title(f"{state} {variable}")
plt.tight_layout()
plt.savefig("notebooks/standardization_mean_gt.png")
plt.show()


In [ ]:
states = ["mean", "gt"]
variables = ["MeanH", "P95", "Gini"]
fig, axs = plt.subplots(nrows=len(variables), ncols=len(states), figsize=(10,6))
for i, variable in enumerate(variables):
    for j, state in enumerate(states):
        if state=="variance": continue
        sub = data.copy()
        sub = sub[sub.variables==variable]
        sub = sub[sub.states==state]
        sns.histplot(data=sub, x="number", hue="sources", kde=True, ax=axs[i,j])
        axs[i,j].set_title(f"{state} {variable}")
fig.suptitle("Projects: {}".format(" ".join(list(data.project_id.unique()))))
plt.tight_layout()
# plt.savefig("notebooks/standardization_ok.png")
plt.show()

In [ ]:
states = ["mean", "gt"]
variables = ["Cover", "Dens"]
fig, axs = plt.subplots(nrows=len(variables), ncols=len(states), figsize=(10,4))
for i, variable in enumerate(variables):
    for j, state in enumerate(states):
        if state=="variance": continue
        sub = data.copy()
        sub = sub[sub.variables==variable]
        sub = sub[sub.states==state]
        sns.histplot(data=sub, x="number", hue="sources", kde=True, ax=axs[i,j])
        axs[i,j].set_title(f"{state} {variable}")
fig.suptitle("Projects: {}".format(" ".join(list(data.project_id.unique()))))
plt.tight_layout()
# plt.savefig("notebooks/standardization_issues.png")
plt.show()

## Trying to understand the issue

In [ ]:
# Are the predictions activated?
with open("config/predict_testset/baseline.yaml", "r") as f:
    pcfg = yaml.safe_load(f)
for i in range(len(pcfg["checkpoint_dirs"])):
    with pjoin(pcfg["checkpoint_dirs"][i], "config.yaml").open("r") as f:
        tmp = yaml.safe_load(f)
        print(
            pcfg["checkpoint_dirs"][i].split("/")[-1],
            "activate_mean:",
            tmp["training"]["activate_mean"]
        )
# yes

In [ ]:
# Show all gt before
variables = ['P95', 'MeanH', 'Dens', 'Gini', 'Cover']
f, axs = plt.subplots(nrows=len(variables), ncols=1, figsize=(8,10))
for i, var in enumerate(variables):
    tmp = data.query(f"states == 'gt' & variables == '{var}' & sources == 'before'")
    sns.histplot(data=tmp, x="number", ax=axs[i])
    axs[i].set_title(f"{var} (min={tmp.number.min():.3f}, max={tmp.number.max():.3f})")
plt.tight_layout()
plt.savefig("notebooks/normalized100_gt_distros.png")
plt.show()

In [ ]:
def show_constraints_violations(df, variable, constraints):
    var = df.query(f"variables == '{variable}' & states != 'variance'")
    var_before = var[var.sources=="before"]
    var_after = var[var.sources=="after"]
    print(variable, "violation rate:")
    for constraint in constraints:
        gt_before_rate = var_before[var_before.states=="gt"].query(constraint).shape[0]/var_before[var_before.states=="gt"].shape[0]
        gt_after_rate = var_after[var_after.states=="gt"].query(constraint).shape[0]/var_after[var_after.states=="gt"].shape[0]
        mean_before_rate = var_before[var_before.states=="mean"].query(constraint).shape[0]/var_before[var_before.states=="mean"].shape[0]
        mean_after_rate = var_after[var_after.states=="mean"].query(constraint).shape[0]/var_after[var_after.states=="mean"].shape[0]
        print(f"\t{constraint}:")
        print(f"\t\tbefore -> (gt={gt_before_rate*100:.2f}%,mean={mean_before_rate*100:.2f}%)")
        print(f"\t\tafter  -> (gt={gt_after_rate*100:.2f}%,mean={mean_after_rate*100:.2f}%)")

In [ ]:
# Are the constraints verified
variables = ['P95', 'MeanH', 'Dens', 'Gini', 'Cover']
for i, var in enumerate(variables):
    show_constraints_violations(
        data,
        var,
        ["number < 0"] if i<2 else ["number < 0", "number > 1"]
    )
# For prediction yes, for gt P95, MeanH, Gini yes Dens, Cover no
# which makes no sense

In [ ]:
# Could it be that for some reason the Cover and Dens GT must be normalized ?
# Compare mean_before with gt_after
import warnings
warnings.simplefilter("ignore")
for var in ["Dens", "Cover"]:
    tmp = data[data.variables==var]
    tmp_mean_before = data.query("states == 'mean' & sources == 'before'")
    tmp_gt_after = data.query("states == 'gt' & sources == 'after'")
    tmp_mean_before["set"] = "mean_before"
    tmp_gt_after["set"] = "gt_after"
    tmp = pd.concat([tmp_mean_before, tmp_gt_after])
    sns.histplot(data=tmp, x="number", hue="set", kde=True)
    plt.title(var)
    plt.show()
# They do seem pretty close... BUT there mean_before respects (0,1) constraint
# while gt_after does not (<0)

In [ ]:
tmp

In [ ]:
1664*480

In [ ]:
gt.shape, gt_mask.shape

In [ ]:
gt[:,gt_mask]

In [ ]:
np.isnan(mean).all(0).shape

In [ ]:
np.isnan(mean).all(axis=0, keepdims=True).shape

In [ ]:
data.query("states == 'gt' & variables == 'P95'").number.unique()

In [ ]:
(gt==-9.99999000e+05).sum()/gt.reshape(-1,).shape[0]

In [ ]:
(gt<0).sum()/gt.reshape(-1,).shape[0]

In [ ]:
with rasterio.open(pjoin(cfg['gt_dir'], f"{project}.tif")) as fh:
    gt = fh.read(fh.indexes)
    gt_mask = fh.read_masks(1).astype(np.bool_)

In [ ]:
([gt==-9.99999000e+05]==gt_mask).sum()/gt_mask.reshape(-1,).shape[0]

In [ ]:
gt.shape, gt_mask.shape

In [ ]:
np.count_nonzero(((gt[0]==-9.99999000e+05)==gt_mask))/(gt_mask.shape[0]*gt_mask.shape[1])

In [ ]:
probably_ok = (gt[0]!=-9.99999000e+05)
definitely_ok = gt_mask
plt.imshow(probably_ok)
plt.show()
plt.imshow(definitely_ok)
plt.show()

In [ ]:
np.logical_and(probably_ok

In [ ]:
definitely_ok[0,0]

In [ ]:
(np.isnan(mean)==([gt==-9.99999000e+05]))